In [51]:
from composable import pipeable
from composable.strict import map, filter
import composable_records as rec
import composable_tuples as tup
from composable_glob import glob
from composable_utility import get, with_open, identity, apply
from composable_object import obj, attr
from composable_origami import fold
from operator import add

### Task 2 - Combine all the `Pitching` yearly files using composable functions

#### Get the `Pitching` paths

In [16]:
(paths :=
 "./data/baseballdatabank_2020_to_2023/**/20*.csv"
 >> glob(recursive=True)
)

['./data/baseballdatabank_2020_to_2023\\contrib\\2020_AwardsManagers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_AwardsPlayers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_AwardsShareManagers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_AwardsSharePlayers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_CollegePlaying.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_HallOfFame.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2020_Salaries.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_AwardsManagers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_AwardsPlayers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_AwardsShareManagers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_AwardsSharePlayers.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_CollegePlaying.csv',
 './data/baseballdatabank_2020_to_2023\\contrib\\2021_HallOfFame.csv',
 './data/baseballdataba

In [17]:
(new_paths := [path.replace("\\", "/") for path in paths])

['./data/baseballdatabank_2020_to_2023/contrib/2020_AwardsManagers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_AwardsPlayers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_AwardsShareManagers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_AwardsSharePlayers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_CollegePlaying.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_HallOfFame.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2020_Salaries.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_AwardsManagers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_AwardsPlayers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_AwardsShareManagers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_AwardsSharePlayers.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_CollegePlaying.csv',
 './data/baseballdatabank_2020_to_2023/contrib/2021_HallOfFame.csv',
 './data/baseballdatabank_2020_to_2023/contrib/20

#### Working pipe

In [52]:
# some useful tools
list(range(10)) >> tup.first

0

In [53]:
list(range(10)) >> tup.rest

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [63]:
(new_paths 
 >> map(rec.create (path = identity,                           # use map - outside is a list[]
                    lines =  with_open(obj.readlines()),       # read in lines, you could use lambda
                   )
       ) 
 >> map(rec.update(len_lines = lambda r: len(r.lines),         # get the length of lines 
                  )
       )
 >> filter(lambda r: r.len_lines > 1)                          # filter out the length bigger than 1 
 >> map(rec.update(file = lambda r: r.path >> obj.split('/') >> get(-1),
                   file_name = lambda r: r.file.split('.') >> get(0),
                   year = lambda r: r.file_name.split('_') >> get(0) >> apply(int), # so that you can filter out in order
                   folder = lambda r: r.path.split('/20') >> get(0) >> apply(lambda s: s + '/'),
                   type_ = lambda r: r.file_name.split('_') >> get(-1),
                   header = lambda r: r.lines >> tup.first, #get(0),
                   body = lambda r: r.lines >> tup.rest,
                   sequential = True
                  )
       )
 >> map(rec.apply(header = lambda s: 'yearID,' + s))
 >> map(rec.update(body = lambda r: r.body >> map(lambda line: str(r.year) + ',' + line),
                  )
       )
 >> rec.group_by('type_')
 >> rec.map(rec.zip_at(['folder', 'header', 'type_', 'body']))
 >> rec.map(rec.apply(header = tup.first,
                      folder = tup.rest,
                      type_ = tup.first, 
                     )
           )
 >> rec.map(rec.update(outpath = lambda r: r['folder'] + r['type_'] + '.csv')) # this is how you access from dict
 >> rec.map(rec.update(body = lambda r: r['body'] >> fold(add, [r['header']]),
                      )
           )
#)
# ) >> map(rec.readable_output(num_keys=10, max_len_seq=5)) # for list[] of records
) >> rec.readable_output(num_keys=5, max_len_seq=3) # for record of lists

TypeError: can only concatenate list (not "str") to list

In [50]:
(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(file   =  get_file,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
) >> map(rec.heads(2))


[{'file': '2022_Pitching.csv',
  'folder': './data/baseballdatabank_2020_to_2023/core/',
  'year': 2022,
  'name': '2022_Pitching',
  'type_': 'Pitching',
  'lines': ['playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
   'abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n'],
  'header': 'playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  'body': ['abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
   'abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n']},
 {'file': '2021_Pitching.csv',
  'folder': './data/baseballdatabank_2020_to_2023/core/',
  'year': 2021,
  'name': '2021_Pitching',
  'type_': 'Pitching',
  'lines': ['playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
   'abadfe01,1,BAL

#### Add the year to the header and each row of the body

**Comprehension code.**
```python
 [record | {'header': 'yearID,' + record['header'],
            'body'  : [str(record['year']) + ',' + line 
                       for line in record['body']
                      ],
           }
     for record in header_and_body
 ]
```
**Functional Code.**
```python
### Access multiple keys ==> UPDATE
(...
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
)
```

In [51]:
add_yearID = pipeable(lambda header: 'yearID,' + header)
add_year_to_line =  pipeable(lambda year, line: str(year) + ',' + line)
                         
(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(path   =  identity,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
) >> map(rec.heads(2))

[{'path': './data/baseballdatabank_2020_to_2023/core/2022_Pitching.csv',
  'folder': './data/baseballdatabank_2020_to_2023/core/',
  'year': 2022,
  'name': '2022_Pitching',
  'type_': 'Pitching',
  'lines': ['playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
   'abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n'],
  'header': 'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  'body': ['2022,abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
   '2022,abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n']},
 {'path': './data/baseballdatabank_2020_to_2023/core/2021_Pitching.csv',
  'folder': './data/baseballdatabank_2020_to_2023/core/',
  'year': 2021,
  'name': '2021_Pitching',
  'type_': 'Pitching',
  'lines': ['playerID,stint,teamID,lgID,W,L,G

#### Zip up the relevant keys.

**Comprehension code.**
```python
  { key: [get(key, rec, None) for rec in with_year_columns]
    for key in ['type', 'folder', 'header', 'body']
  }
```
**Functional Code.**
```python
### Combine all values at the given keys ==> ZIP AT
(...
 >> rec.zip_at(['type_', 'folder', 'header', 'body'])
)
```

In [52]:
(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(path   =  identity,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
 >> rec.zip_at(['type_', 'folder', 'header', 'body'])
) >> rec.apply(body = map(tup.head(2)))

{'type_': ['Pitching', 'Pitching', 'Pitching'],
 'folder': ['./data/baseballdatabank_2020_to_2023/core/',
  './data/baseballdatabank_2020_to_2023/core/',
  './data/baseballdatabank_2020_to_2023/core/'],
 'header': ['yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n'],
 'body': [['2022,abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
   '2022,abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n'],
  ['2021,abadfe01,1,BAL,AL,0,0,16,0,0,0,0,53,23,11,1,7,10,0.307,5.6,1,0,0,0,82,4,12,0,0,3\n',
   '2021,abbotco01,1,CHN,NL,0,0,7,1,0,0,0,52,20,13,7,11,12,0.286,6.75,0,2,0,0,82,2,15,1,0,0\n'],
  ['2020,abreual01,

#### Extract the folder, type, and header

**Comprehension code.**
```python
  zipped_batting | {'folder':zipped_batting['folder'][0],
                    'type':zipped_batting['type'][0],
                    'header':zipped_batting['header'][0],
                   } 
```
**Functional Code.**
```python
### Only need value at given key ==> APPLY
(...
 >> rec.apply(folder = tup.first,
              type_ = tup.first,
              header = tup.first,
             )
)
```

In [53]:
(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(path   =  identity,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
 >> rec.zip_at(['type_', 'folder', 'header', 'body'])
 >> rec.apply(folder = tup.first,
              type_ = tup.first,
              header = tup.first,
             )
) >> rec.apply(body = map(tup.head(2)))


{'type_': 'Pitching',
 'folder': './data/baseballdatabank_2020_to_2023/core/',
 'header': 'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
 'body': [['2022,abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
   '2022,abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n'],
  ['2021,abadfe01,1,BAL,AL,0,0,16,0,0,0,0,53,23,11,1,7,10,0.307,5.6,1,0,0,0,82,4,12,0,0,3\n',
   '2021,abbotco01,1,CHN,NL,0,0,7,1,0,0,0,52,20,13,7,11,12,0.286,6.75,0,2,0,0,82,2,15,1,0,0\n'],
  ['2020,abreual01,1,NYA,AL,0,1,2,0,0,0,0,4,4,3,1,2,2,0.5,20.25,0,0,1,0,11,1,4,0,0,0\n',
   '2020,abreubr01,1,HOU,AL,0,0,4,0,0,0,0,10,1,1,0,7,3,0.091,2.7,0,0,2,0,20,1,2,0,0,0\n']]}

#### Fold the bodies starting with the header

**Comprehension code.**
```python
  combine_file_info | {'body':fold(lambda acc, el: acc + el,
                                [combine_file_info['header']],
                                combine_file_info['body']
                               )
                        } 
```
**Functional Code.**
```python
### Access multiple keys ==> UPDATE
(...
 >> rec.update(body = lambda r: r >> get('body') >> fold(add, [r['header']])
              )
)
```

In [55]:
(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(path   =  identity,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
 >> rec.zip_at(['type_', 'folder', 'header', 'body'])
 >> rec.apply(folder = tup.first,
              type_ = tup.first,
              header = tup.first,
             )
 >> rec.update(body = lambda r: r >> get('body') >> fold(add, [r['header']])
              )
) >> rec.apply(body = tup.head(5))


{'type_': 'Pitching',
 'folder': './data/baseballdatabank_2020_to_2023/core/',
 'header': 'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
 'body': ['yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  '2022,abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
  '2022,abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n',
  '2022,abreual01,2,KCA,AL,0,0,4,0,0,0,0,13,6,2,1,4,3,0.353,4.15,0,1,1,0,22,3,2,0,0,2\n',
  '2022,abreual01,3,NYA,AL,2,2,22,0,0,0,0,77,25,9,2,6,26,0.255,3.16,0,5,1,0,108,8,10,1,2,2\n']}

#### Make the output path

**Comprehension code.**
```python
  combined_body | {'out_path':combined_body['folder'] + combined_body['type'] + '.csv',
                   } 
```

**Functional Code.**
```python
get_out_path = lambda r: r['folder'] + r['type_'] + '.csv'

### Create a new pair ==> UPDATE
(...
 >> rec.update(out_path = get_out_path)
)
```

In [56]:
get_out_path = lambda r: r['folder'] + r['type_'] + '.csv'

(pitching_files :=
 "./data/baseballdatabank_2020_to_2023/**/20*_Pitching.csv"
 >> glob(recursive=True)
 >> map(rec.create(path   =  identity,
                   folder =  get_folder,
                   year   =  get_year,
                   name   =  get_name,
                   type_  =  get_type,
                   lines  =  with_open(obj.readlines()),
                   use_record_class=False
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('lines') >> tup.first,
                   body = lambda r: r >> get('lines') >> tup.rest,
                  )
       )
 >> map(rec.update(header = lambda r: r >> get('header') >> add_yearID,
                   body = lambda r: r >> get('body') >> map(add_year_to_line(r['year'])),
                  )
       )
 >> rec.zip_at(['type_', 'folder', 'header', 'body'])
 >> rec.apply(folder = tup.first,
              type_ = tup.first,
              header = tup.first,
             )
 >> rec.update(body = lambda r: r >> get('body') >> fold(add, [r['header']])
              )
 >> rec.update(out_path = get_out_path)
) >> rec.apply(body = tup.head(5))


{'type_': 'Pitching',
 'folder': './data/baseballdatabank_2020_to_2023/core/',
 'header': 'yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
 'body': ['yearID,playerID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP\n',
  '2022,abbotco01,1,WAS,NL,0,5,16,9,0,0,0,144,44,28,12,25,45,0.239,5.25,0,2,5,0,216,7,30,0,2,0\n',
  '2022,abreual01,1,TEX,AL,0,0,7,0,0,0,0,26,4,3,2,12,9,0.138,3.12,0,0,1,0,42,0,3,0,0,1\n',
  '2022,abreual01,2,KCA,AL,0,0,4,0,0,0,0,13,6,2,1,4,3,0.353,4.15,0,1,1,0,22,3,2,0,0,2\n',
  '2022,abreual01,3,NYA,AL,2,2,22,0,0,0,0,77,25,9,2,6,26,0.255,3.16,0,5,1,0,108,8,10,1,2,2\n'],
 'out_path': './data/baseballdatabank_2020_to_2023/core/Pitching.csv'}